In [17]:
import pandas as pd
import numpy as np
from data import TASK3_LABELS

In [18]:
TASK3_LABELS

['Appeal_to_Authority',
 'Appeal_to_Fear-Prejudice',
 'Appeal_to_Hypocrisy',
 'Appeal_to_Popularity',
 'Appeal_to_Time',
 'Appeal_to_Values',
 'Causal_Oversimplification',
 'Consequential_Oversimplification',
 'Conversation_Killer',
 'Doubt',
 'Exaggeration-Minimisation',
 'False_Dilemma-No_Choice',
 'Flag_Waving',
 'Guilt_by_Association',
 'Loaded_Language',
 'Name_Calling-Labeling',
 'Obfuscation-Vagueness-Confusion',
 'Questioning_the_Reputation',
 'Red_Herring',
 'Repetition',
 'Slogans',
 'Straw_Man',
 'Whataboutism']

In [19]:
def to_matrix(df3):    
    df3 = df3.where(pd.notnull(df3), '')
    labels_matrix = np.zeros((df3.shape[0], len(TASK3_LABELS)),int)
    
    for idx, label in enumerate(TASK3_LABELS):
        labels_matrix[:, idx] = df3.labels.apply(lambda x: 1 if label in x else 0)
    return labels_matrix

In [37]:
# removed false dillema training sze with dev is 185, under 150:
id_fews = [2,3,4,5,7,8,13,16,17,18,21,22]
id2label = {idx:label for idx, label in enumerate(TASK3_LABELS)}
np.array(TASK3_LABELS)[id_fews]

array(['Appeal_to_Hypocrisy', 'Appeal_to_Popularity', 'Appeal_to_Time',
       'Appeal_to_Values', 'Consequential_Oversimplification',
       'Conversation_Killer', 'Guilt_by_Association',
       'Obfuscation-Vagueness-Confusion', 'Questioning_the_Reputation',
       'Red_Herring', 'Straw_Man', 'Whataboutism'], dtype='<U32')

In [38]:
df_mul = pd.read_csv('Test_task3_en_xlm.txt', sep='\t', header=None, names=['id','line','labels'])
#df_anchor = pd.read_csv('anchor_small_task3_en.txt', sep='\t', header=None, names=['id','line','labels']) #majority_pred_en.csv
df_rob = pd.read_csv('Test_task3_en_rob.txt', sep='\t', header=None, names=['id','line','labels'])
#df_anchor_s = pd.read_csv('majority_pred_en.csv', sep='\t', header=None, names=['id','line','labels'])
matrix_mul1 = to_matrix(df_mul)
matrix_rob1 = to_matrix(df_rob)

In [39]:
sum(matrix_mul1)

array([ 44,  86,   3,  13,   0,   0,  26,   0,   4,  92,  61,  21,  84,
         3, 311, 145,   1,   0,   6, 116,  31,   0,   0])

In [40]:
sum(matrix_rob1)

array([  7,  65,   0,   3,   0,   0,  12,   0,  13,  94,  59,  10,  75,
         1, 287, 101,   0,   0,   0, 103,  19,   0,   0])

In [41]:
sum(matrix_rob1>0)[id_fews]

array([ 0,  3,  0,  0,  0, 13,  1,  0,  0,  0,  0,  0])

In [42]:
sum(matrix_mul1>0)[id_fews]

array([ 3, 13,  0,  0,  0,  4,  3,  1,  0,  6,  0,  0])

In [43]:
matrix_com = np.zeros((df_rob.shape[0], len(TASK3_LABELS)),int)
for idx in range(0,23):
    for j in range(0,matrix_com.shape[0]):
        if idx in id_fews:
            matrix_com[j,idx] = matrix_mul1[j,idx]
        else:
            matrix_com[j,idx] = matrix_rob1[j,idx]

In [44]:
sum(matrix_com == matrix_rob1)

array([910, 910, 907, 898, 910, 910, 910, 910, 897, 910, 910, 910, 910,
       906, 910, 910, 909, 910, 904, 910, 910, 910, 910])

In [45]:
sum(matrix_com == matrix_mul1)

array([867, 835, 910, 910, 910, 910, 880, 910, 910, 804, 854, 887, 881,
       910, 758, 820, 910, 910, 910, 765, 886, 910, 910])

In [46]:
pred_com=pd.Series(list(matrix_com))
series_labels=pred_com.apply(lambda x: [id2label[i] for i in np.where(x==1)[0]])
df_rob['labels_pred']=series_labels.apply(lambda x: ','.join(x))
output_pred_file= 'Test_task3_en.txt'
        
df_rob[['id','line','labels_pred']].to_csv(output_pred_file, sep='\t', header=None, index=None)

In [ ]:
# tjek the new
df_com = pd.read_csv(output_pred_file, sep='\t', header=None, names=['id','line','labels'])
matrix_com1 = to_matrix(df_com)
sum(matrix_com1)